In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
from ipywidgets import *
import plotly.graph_objs as go
import numpy as np


In [2]:
Noutcomes=8

In [3]:
 def process_data(Probs_text, Outcomes_text, RN_text):
    Probs=np.fromstring(Probs_text.value, dtype=float, sep=',')
    CDF= np.append(0,np.cumsum(Probs))
    x=np.repeat(0,len(CDF))
    Outcomes=np.fromstring(Outcomes_text.value, dtype=float, sep=',')
    RN=np.fromstring(RN_text.value, dtype=float, sep=',')[0]
    return Probs, CDF,x,Outcomes,RN
 
def binary_search(CDF, Outcomes, RN,search_rn):
    CDF_sub=CDF[1:]
    low, high = 0, len(CDF_sub)-1
    i = 0
    while low < high :
        i = i+1
        if i>search_rn:
            break
        mid = int((low + high)/ 2)
        if  RN < CDF_sub[mid] :
            high = mid 
        else: 
            low = mid + 1  
    colors=np.zeros(len(CDF))
    colors[low:high+1]=1
    return Outcomes[mid], colors, i, low, high, mid

 
# Assign an empty figure widget with two traces
 

def initialize(): 
    values=np.arange(5,5+Noutcomes)
    values_str=str(values[0])
    for i in values[1:]:
        values_str+=","+str(i)
    probs=np.round(np.diff(np.logspace(0,1,len(values))/10,prepend=0),2)
    probs[-1]=1-np.sum(probs[:-1])
    probs_str=str(probs[0])
    for i in probs[1:]:
        probs_str+=","+str(i)
    Outcomes_text=widgets.Text(
        value=values_str  ,
        description='outcome value:',
        disabled=False
    )

    Probs_text=widgets.Text(
        value=probs_str,
        description='prob value:',
        disabled=False
    )
    FB=widgets.HTML(
        value="",
        description='',
        disabled=False
    )

    RNG=widgets.Button(description="Generate RN")
    update=widgets.Button(description="new distribution")

    RN_text=widgets.Text(
        value="0.6111",
        description='a value:',
        disabled=False
    )

    Probs,CDF,x,Outcomes,RN=process_data(Probs_text, Outcomes_text, RN_text)
    _,_,Nsearches,i,_,_=binary_search(CDF, Outcomes, float(RN_text.value),search_rn=100000)
   
    search_rn=widgets.IntSlider(
        value=0,
        min=0,
        max=Nsearches,
        step=1,
        description='Search',
        disabled=False,
        indent=False
    )

    high=len(Outcomes)-1
    low=0
    mid=int((high+low)/2)
    Description=widgets.HTML(
        value='init: low=0, high='+str(high)+", mid="+str(mid),
        disabled=False
    )

     
    Outcomes_index=[str(Outcomes[i])+" index:"+str(i) for i in range(len(Outcomes))]
    
    Outcomes_index=np.array(Outcomes_index)
    trace1=go.Bar(x=x,y=Probs, marker=dict(color=CDF,colorscale="PuBu"), 
                  hoverinfo="text",hovertext =Outcomes_index,name="Outcome regions"+' '*20   )
     
    trace2=go.Scatter(x=[0],y=[RN],hoverinfo="text",text="u="+str(RN)+"<br>converts to "+str(Outcomes[i]),
                      marker=dict(size=20,color="red"),name="Random number")
    trace3=go.Scatter(x=x,y=CDF,mode='markers+lines+text', hoverinfo="skip", text=np.round(CDF,2),
                      textposition="middle left",marker=dict(size=20,symbol="line-ew-open")
                      ,marker_color="black",name="CDF")
    trace4=go.Scatter(x=x[1:i+1],y=CDF[1:i+1],mode='markers', hoverinfo="text",
                      textposition="middle left",marker=dict(size=9,symbol="x")
                      ,marker_color="blue",text="u>=cdf, move above",name="Failed comparisons")
    trace5=go.Scatter(x=x[[i+1]],y=CDF[[i+1]],mode='markers', hoverinfo="text",
                      textposition="middle left",marker=dict(size=9,symbol="circle")
                      ,name="Successful comparison",marker_color="blue",text="u<cdf, stop")

    g = go.FigureWidget(data=[trace1,trace2,trace3],
                        layout=go.Layout(
                            title=dict(
                            text=''
                            )
                            ,xaxis = dict(title = "Inverse Transform Method", range = [-0.5,0.5],showticklabels=False),
                            yaxis = dict(title = "", range = [-.1,1.1]),
                            hovermode=None,
                            margin={'l': 0, 'r': 0, 't': 0, 'b': 0},width=600, height=400 

                        ))

    return Outcomes_text,Probs_text, RNG, update, search_rn, RN_text, FB,g,trace4,trace5, Description
Outcomes_text,Probs_text, RNG, update, search_rn, RN_text, FB,g,trace4,trace5, Description=initialize()
 
Probs,CDF,x,Outcomes,RN=process_data(Probs_text, Outcomes_text, RN_text)
#i=convert(Outcomes,CDF,RN) 
    
def response(change):
    Probs,CDF,x,Outcomes,RN=process_data(Probs_text, Outcomes_text, RN_text)
    if len(Probs)!=len(Outcomes):
        print(Probs,Outcomes)
        FB.value="<b>WARNING: # Outcomes and # Probs are different  </b>"
    elif np.isclose(np.sum(Probs),1)==0:
        print(np.sum(Probs))
        FB.value="<b>WARNING:Probs do not sum up to 1 </b>"
    elif np.min(Probs)<0:
        FB.value="<b>WARNING:Contains negative value</b>"
    elif len(np.unique(Outcomes))!=len(Probs):
        FB.value="<b>WARNING:Contains repetitive outcomes</b>"
    else:
        FB.value=""
        with g.batch_update():
            _,_,Nsearches,i,_,_=binary_search(CDF, Outcomes, float(RN_text.value),search_rn=100000)
            search_rn.max=Nsearches
            g.data[0].x = x
            g.data[0].y = Probs
            g.data[0].marker.color=CDF
            Outcomes_index=[str(Outcomes[i])+" index:"+str(i) for i in range(len(Outcomes))]
            g.data[0].hovertext=Outcomes_index
             
            g.data[1].y=[RN]
            
            g.data[1].text="u="+str(RN)+"<br>converts to "+str(Outcomes[i])
            g.data[2].x = x
            g.data[2].y =CDF
            g.data[2].text=np.round(CDF,2)
            search_rn.value=0
            if search_rn.value>0:
                i=convert(Outcomes,CDF,RN)  
 

def response2(change):
    if search_rn.value>=1:
        with g.batch_update():
            Probs,CDF,x,Outcomes,RN=process_data(Probs_text, Outcomes_text, RN_text)
            output,color_scheme,Nsearches,low,high,mid=binary_search(CDF, Outcomes, RN,search_rn.value)
            
            g.data[0].marker.color=color_scheme
            mid=int((low+high)/2)
            #Reason="since random number 
            Description.value="low="+str(low)+", high="+str(high)+", mid="+str(mid)
            #i=convert(Outcomes,CDF,RN)   
 
    else:
        Probs,CDF,x,Outcomes,RN=process_data(Probs_text, Outcomes_text, RN_text)
        high=len(Outcomes)-1
        low=0
        mid=int((high+low)/2)
        Description.value='init: low=0, high='+str(high)+", mid="+str(mid)

        g.data[0].marker.color=CDF
        g.data = [g.data[0], g.data[1],g.data[2]]

def change_random(b):
    RN_text.value=str(np.random.rand())

RNG.on_click(change_random )
update.on_click(response)
search_rn.observe(response2,names="value")
RN_text.observe(response,names="value")
  
        
container1 = widgets.HBox([RNG,RN_text])
container2 = widgets.HBox([Outcomes_text,Probs_text,update])
container4 = widgets.HBox([search_rn])
container5 = widgets.HBox([Description])
container3 = widgets.HBox([FB])

 

Widget1=widgets.VBox([container1,container2,container3,container4, container5,g])
Widget1
